# GPT-2 Fine-Tuning with Elastic Training

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mmcmanus1/elastic-training-harness/blob/main/notebooks/elastic_training_gpt2_finetune.ipynb)

A complete fine-tuning pipeline that produces **coherent text output**:
- **Pretrained GPT-2** (124M parameters) as starting point
- **WikiText-2 dataset** for domain adaptation
- **1000 training steps** with validation tracking
- **Full elastic_harness features** - checkpointing, scaling, recovery

Unlike the basic demo notebooks that train from scratch (and produce gibberish), this notebook fine-tunes a pretrained model to demonstrate a full training pipeline with coherent results.

Designed for Google Colab T4 GPU (16GB VRAM).

## 1. Setup

Clone the repository and install dependencies.

In [ ]:
# Setup: Clone and install elastic-training-harness
# Clean up any existing installation (safe to re-run)
!rm -rf /content/elastic-training-harness

# Clone the repository
!git clone https://github.com/mmcmanus1/elastic-training-harness.git

# Install the package and additional dependencies (non-editable for Colab compatibility)
%cd /content/elastic-training-harness
!pip install . --quiet
!pip install datasets transformers --quiet
%cd /content

# Force Python to recognize newly installed package
import importlib
import sys
if 'elastic_harness' in sys.modules:
    del sys.modules['elastic_harness']
importlib.invalidate_caches()

# Verify installation
try:
    import elastic_harness
    print(f"elastic_harness {elastic_harness.__version__} installed successfully!")
except ImportError as e:
    print(f"Installation failed: {e}")
    print("Try: Runtime -> Restart runtime, then run this cell again")

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import time
import random
import os
import math

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {gpu_memory:.1f} GB")

## 2. Import elastic_harness Components

In [ ]:
# Checkpoint management
from elastic_harness.checkpoint import (
    CheckpointConfig,
    CheckpointState,
    CheckpointManager,
    CheckpointTier,
    MemorySnapshotBackend,
    NVMeBackend,
)

# LR scaling and gradient accumulation
from elastic_harness.scaling import (
    ScalingRule,
    ScalingConfig,
    LRScalingManager,
    GradAccumulationConfig,
    GradientAccumulationManager,
    ElasticScalingManager,
)

print("elastic_harness components imported successfully!")

## 3. Load Pretrained GPT-2

Load GPT-2 (124M parameters) from Hugging Face. This model was pretrained on ~40GB of internet text.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pretrained GPT-2
print("Loading pretrained GPT-2...")
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
num_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model parameters: {num_params:,}")
print(f"Trainable parameters: {num_trainable:,}")
print(f"Model size: ~{num_params * 4 / 1e6:.1f} MB (fp32)")
print(f"Vocab size: {tokenizer.vocab_size}")

## 4. Load WikiText-2 Dataset

Fine-tune on WikiText-2 to adapt the model to Wikipedia-style text.

In [ ]:
from datasets import load_dataset

# Load WikiText-2
print("Loading WikiText-2 dataset...")
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

print(f"Train samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")

In [ ]:
SEQ_LENGTH = 256

class WikiTextDataset(Dataset):
    """Dataset that tokenizes and chunks text into fixed-length sequences."""
    
    def __init__(self, texts, tokenizer, seq_length):
        self.seq_length = seq_length
        
        # Concatenate all text and tokenize
        all_text = " ".join([t for t in texts if t.strip()])
        tokens = tokenizer.encode(all_text, add_special_tokens=False)
        
        # Chunk into sequences
        self.chunks = []
        for i in range(0, len(tokens) - seq_length, seq_length):
            self.chunks.append(tokens[i:i + seq_length])
        
        print(f"Created {len(self.chunks)} sequences of length {seq_length}")
    
    def __len__(self):
        return len(self.chunks)
    
    def __getitem__(self, idx):
        chunk = torch.tensor(self.chunks[idx], dtype=torch.long)
        # For causal LM, input and labels are the same (model handles shifting)
        return chunk, chunk

# Create datasets
print("Tokenizing and chunking train data...")
train_dataset = WikiTextDataset(
    dataset['train']['text'], 
    tokenizer, 
    SEQ_LENGTH
)

print("Tokenizing and chunking validation data...")
val_dataset = WikiTextDataset(
    dataset['validation']['text'],
    tokenizer,
    SEQ_LENGTH
)

In [ ]:
# Create DataLoaders
BATCH_SIZE = 2  # Small batch, will use gradient accumulation

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

## 5. Text Generation Function

Define a function to generate text and track improvement during training.

In [ ]:
@torch.no_grad()
def generate_samples(model, tokenizer, prompts, device, max_new_tokens=50, temperature=0.8):
    """Generate text samples from the model."""
    model.eval()
    results = []
    
    for prompt in prompts:
        input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
        
        output = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
        
        generated = tokenizer.decode(output[0], skip_special_tokens=True)
        results.append((prompt, generated))
    
    model.train()
    return results


def print_samples(samples, step_label):
    """Pretty print generated samples."""
    print(f"\n{'='*60}")
    print(f"Text Generation - {step_label}")
    print('='*60)
    for prompt, generated in samples:
        print(f"\nPrompt: {prompt}")
        print(f"Generated: {generated}")


# Test prompts we'll use throughout training
TEST_PROMPTS = [
    "The history of",
    "In the year 2024,",
    "Scientists discovered that",
]

## 6. Baseline Generation (Before Fine-Tuning)

Generate text with the pretrained model before any fine-tuning.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Generate baseline samples
baseline_samples = generate_samples(model, tokenizer, TEST_PROMPTS, device)
print_samples(baseline_samples, "Pretrained Baseline (Step 0)")

## 7. Training with Full elastic_harness Features

Fine-tuning loop with:
- Gradient accumulation (effective batch size 8)
- Multi-tier checkpointing (Memory + NVMe)
- LR warmup with scaling
- Validation and perplexity tracking
- Text generation checkpoints

In [ ]:
def get_gpu_memory_mb():
    """Get current GPU memory usage in MB."""
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1e6
    return 0


@torch.no_grad()
def evaluate(model, val_loader, device, max_batches=50):
    """Run validation and return average loss and perplexity."""
    model.eval()
    total_loss = 0
    num_batches = 0
    
    for i, (input_ids, labels) in enumerate(val_loader):
        if i >= max_batches:
            break
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        
        outputs = model(input_ids=input_ids, labels=labels)
        total_loss += outputs.loss.item()
        num_batches += 1
    
    model.train()
    avg_loss = total_loss / num_batches if num_batches > 0 else 0
    perplexity = math.exp(avg_loss) if avg_loss < 10 else float('inf')
    return avg_loss, perplexity


def finetune_gpt2(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    tokenizer,
    max_steps: int = 1000,
    accumulation_steps: int = 4,  # Effective batch = 2 * 4 = 8
    checkpoint_interval: int = 200,
    memory_snapshot_interval: int = 25,
    val_interval: int = 100,
    log_interval: int = 25,
    generation_interval: int = 500,
    chaos_enabled: bool = False,
    chaos_probability: float = 0.01,
    chaos_after_step: int = 300,
):
    """Fine-tuning loop with full elastic_harness features."""

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Setup optimizer with fine-tuning learning rate
    optimizer = torch.optim.AdamW(
        model.parameters(), 
        lr=5e-5,  # Lower LR for fine-tuning
        weight_decay=0.01,
        betas=(0.9, 0.999),
    )

    # Setup elastic scaling manager (unified LR + accumulation)
    lr_config = ScalingConfig(
        base_lr=5e-5,
        base_batch_size=BATCH_SIZE * accumulation_steps,
        base_world_size=1,
        scaling_rule=ScalingRule.SQRT,
        warmup_steps=100,  # Longer warmup for fine-tuning
    )
    lr_manager = LRScalingManager(lr_config, optimizer)

    # Setup multi-tier checkpoint backends
    memory_backend = MemorySnapshotBackend(max_snapshots=3)
    nvme_backend = NVMeBackend(base_path='/tmp/gpt2_finetune_checkpoints')

    # Try to resume from checkpoint
    start_step = 0
    latest_path = nvme_backend.get_latest_checkpoint()
    if latest_path:
        checkpoint = nvme_backend.load(latest_path)
        model.load_state_dict(checkpoint.model_state_dict)
        optimizer.load_state_dict(checkpoint.optimizer_state_dict)
        start_step = checkpoint.step + 1
        print(f"Resumed from checkpoint at step {checkpoint.step}")
    else:
        print("Starting fresh fine-tuning run")

    # Training state
    train_losses = []
    val_losses = []
    val_perplexities = []
    val_steps = []
    memory_usage = []
    generation_samples = {}  # Store samples at different steps
    
    model.train()
    train_iter = iter(train_loader)
    accumulated_loss = 0
    
    print(f"\nStarting fine-tuning from step {start_step} to {max_steps}")
    print(f"Effective batch size: {BATCH_SIZE * accumulation_steps}")
    print(f"Learning rate: 5e-5 (with {lr_config.warmup_steps}-step warmup)")
    print(f"Chaos mode: {'ENABLED' if chaos_enabled else 'disabled'}")
    print("-" * 60)

    for step in range(start_step, max_steps):
        step_start = time.time()

        # Chaos: maybe crash (for testing recovery)
        if chaos_enabled and step > chaos_after_step:
            if random.random() < chaos_probability:
                print(f"\nCHAOS: Simulated crash at step {step}!")
                print("Run this cell again to resume from checkpoint.")
                return {
                    'train_losses': train_losses,
                    'val_losses': val_losses,
                    'val_perplexities': val_perplexities,
                    'val_steps': val_steps,
                    'memory_usage': memory_usage,
                    'generation_samples': generation_samples,
                    'final_step': step,
                    'crashed': True,
                }

        # Memory snapshot
        if step > 0 and step % memory_snapshot_interval == 0:
            state = CheckpointState(
                step=step,
                model_state_dict=model.state_dict(),
                optimizer_state_dict=optimizer.state_dict(),
            )
            memory_backend.save(state)

        # Gradient accumulation loop
        optimizer.zero_grad()
        for micro_step in range(accumulation_steps):
            try:
                input_ids, labels = next(train_iter)
            except StopIteration:
                train_iter = iter(train_loader)
                input_ids, labels = next(train_iter)
            
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss / accumulation_steps
            loss.backward()
            accumulated_loss += loss.item()

        # Gradient clipping and optimizer step
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # LR warmup step
        current_lr = lr_manager.step()

        train_losses.append(accumulated_loss)
        memory_usage.append(get_gpu_memory_mb())
        accumulated_loss = 0

        # Logging
        if step % log_interval == 0:
            avg_loss = sum(train_losses[-log_interval:]) / min(log_interval, len(train_losses))
            ppl = math.exp(avg_loss) if avg_loss < 10 else float('inf')
            mem_mb = memory_usage[-1]
            step_time = time.time() - step_start
            tokens_per_sec = BATCH_SIZE * accumulation_steps * SEQ_LENGTH / step_time
            print(
                f"Step {step:4d} | Loss: {avg_loss:.4f} | PPL: {ppl:.1f} | "
                f"LR: {current_lr:.2e} | Mem: {mem_mb:.0f}MB | "
                f"Tok/s: {tokens_per_sec:.0f}"
            )

        # Validation
        if step > 0 and step % val_interval == 0:
            val_loss, val_ppl = evaluate(model, val_loader, device)
            val_losses.append(val_loss)
            val_perplexities.append(val_ppl)
            val_steps.append(step)
            print(f"  [Validation loss: {val_loss:.4f}, perplexity: {val_ppl:.1f}]")

        # Generation checkpoint
        if step > 0 and step % generation_interval == 0:
            samples = generate_samples(model, tokenizer, TEST_PROMPTS, device)
            generation_samples[step] = samples
            print_samples(samples, f"Step {step}")
            model.train()  # Ensure back in training mode

        # NVMe checkpoint
        if step > 0 and step % checkpoint_interval == 0:
            state = CheckpointState(
                step=step,
                model_state_dict=model.state_dict(),
                optimizer_state_dict=optimizer.state_dict(),
                metrics={'train_loss': train_losses[-1], 'val_loss': val_losses[-1] if val_losses else None},
            )
            path = f"checkpoint_step_{step:08d}.pt"
            nvme_backend.save(state, path)
            nvme_backend.cleanup_old_checkpoints(keep_last=2)
            print(f"  [Checkpoint saved: {path}]")
            memory_backend.clear()

    # Final validation
    final_val_loss, final_val_ppl = evaluate(model, val_loader, device)
    val_losses.append(final_val_loss)
    val_perplexities.append(final_val_ppl)
    val_steps.append(max_steps)

    # Final generation
    final_samples = generate_samples(model, tokenizer, TEST_PROMPTS, device)
    generation_samples[max_steps] = final_samples

    print("-" * 60)
    print(f"Fine-tuning complete!")
    print(f"Final train loss: {train_losses[-1]:.4f}")
    print(f"Final val loss: {final_val_loss:.4f}")
    print(f"Final perplexity: {final_val_ppl:.1f}")

    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'val_perplexities': val_perplexities,
        'val_steps': val_steps,
        'memory_usage': memory_usage,
        'generation_samples': generation_samples,
        'final_step': max_steps,
        'crashed': False,
    }

## 8. Run Fine-Tuning

Fine-tune for 1000 steps. Set `chaos_enabled=True` to test crash recovery.

In [ ]:
# Run fine-tuning
results = finetune_gpt2(
    model,
    train_loader,
    val_loader,
    tokenizer,
    max_steps=1000,
    accumulation_steps=4,  # Effective batch = 8
    checkpoint_interval=200,
    memory_snapshot_interval=25,
    val_interval=100,
    log_interval=25,
    generation_interval=500,
    chaos_enabled=False,  # Set True to test recovery
    chaos_probability=0.02,
    chaos_after_step=300,
)

## 9. Visualize Training Progress

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Training loss
axes[0, 0].plot(results['train_losses'], alpha=0.7)
axes[0, 0].set_xlabel('Step')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training Loss')
axes[0, 0].grid(True, alpha=0.3)

# Validation loss
axes[0, 1].plot(results['val_steps'], results['val_losses'], 'o-', color='orange')
axes[0, 1].set_xlabel('Step')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].set_title('Validation Loss')
axes[0, 1].grid(True, alpha=0.3)

# Perplexity
axes[1, 0].plot(results['val_steps'], results['val_perplexities'], 's-', color='green')
axes[1, 0].set_xlabel('Step')
axes[1, 0].set_ylabel('Perplexity')
axes[1, 0].set_title('Validation Perplexity')
axes[1, 0].grid(True, alpha=0.3)

# Memory usage
axes[1, 1].plot(results['memory_usage'], color='purple', alpha=0.7)
axes[1, 1].set_xlabel('Step')
axes[1, 1].set_ylabel('Memory (MB)')
axes[1, 1].set_title('GPU Memory Usage')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Initial loss: {results['train_losses'][0]:.4f}")
print(f"Final loss: {results['train_losses'][-1]:.4f}")
print(f"Loss reduction: {results['train_losses'][0] - results['train_losses'][-1]:.4f}")
if results['val_perplexities']:
    print(f"Final perplexity: {results['val_perplexities'][-1]:.1f}")

## 10. Final Text Generation Comparison

Compare text generation before and after fine-tuning.

In [ ]:
print("="*70)
print("TEXT GENERATION COMPARISON: Before vs After Fine-Tuning")
print("="*70)

print("\n" + "-"*70)
print("BASELINE (Pretrained GPT-2, before fine-tuning)")
print("-"*70)
for prompt, generated in baseline_samples:
    print(f"\nPrompt: {prompt}")
    print(f"Generated: {generated}")

print("\n" + "-"*70)
print(f"AFTER FINE-TUNING (Step {results['final_step']})")
print("-"*70)
final_samples = results['generation_samples'].get(results['final_step'], [])
for prompt, generated in final_samples:
    print(f"\nPrompt: {prompt}")
    print(f"Generated: {generated}")

print("\n" + "="*70)
print("The fine-tuned model adapts to Wikipedia-style text from WikiText-2.")
print("You may notice more formal, encyclopedic language patterns.")
print("="*70)

## 11. Checkpoint Recovery Demo

Demonstrate loading from a saved checkpoint and verifying model state.

In [ ]:
print("Checkpoint Recovery Demonstration")
print("=" * 50)

# Create a fresh model
fresh_model = GPT2LMHeadModel.from_pretrained("gpt2")

# Load checkpoint
nvme_backend = NVMeBackend(base_path='/tmp/gpt2_finetune_checkpoints')
latest_path = nvme_backend.get_latest_checkpoint()

if latest_path:
    checkpoint = nvme_backend.load(latest_path)
    fresh_model.load_state_dict(checkpoint.model_state_dict)
    
    print(f"Loaded checkpoint from step {checkpoint.step}")
    print(f"Checkpoint metrics: {checkpoint.metrics}")
    
    # Verify model works
    fresh_model = fresh_model.to(device)
    
    val_loss, val_ppl = evaluate(fresh_model, val_loader, device)
    print(f"Validation loss after loading: {val_loss:.4f}")
    print(f"Validation perplexity: {val_ppl:.1f}")
    
    # Generate text to verify
    print("\nGeneration test after checkpoint load:")
    samples = generate_samples(fresh_model, tokenizer, ["The history of"], device, max_new_tokens=30)
    print(f"Generated: {samples[0][1]}")
    
    print("\nCheckpoint recovery successful!")
else:
    print("No checkpoint found - run training first")

## 12. Topology Change Simulation

Demonstrate how elastic_harness handles dynamic world size changes.

In [ ]:
print("Simulating Topology Changes")
print("=" * 60)

# Setup with elastic scaling managers (using a tiny model for demo)
dummy_model = GPT2LMHeadModel.from_pretrained("gpt2")
dummy_optimizer = torch.optim.AdamW(dummy_model.parameters(), lr=5e-5)

# ElasticScalingManager combines LR and accumulation
elastic_manager = ElasticScalingManager(
    optimizer=dummy_optimizer,
    scaling_config=ScalingConfig(
        base_lr=5e-5,
        base_batch_size=8,
        base_world_size=4,
        scaling_rule=ScalingRule.SQRT,
        warmup_steps=100,
    ),
    accum_config=GradAccumulationConfig(
        target_global_batch_size=512,
        local_batch_size=8,
        base_world_size=4,
    ),
)

print(f"Initial state (4 GPUs):")
print(f"  LR: {elastic_manager.current_lr:.2e}")
print(f"  Accumulation steps: {elastic_manager.accumulation_steps}")
print(f"  Global batch: {8 * 4 * elastic_manager.accumulation_steps}")
print()

# Simulate losing 2 GPUs
print("Worker failure: 4 GPUs -> 2 GPUs")
elastic_manager.on_topology_change(2)
print(f"  New LR: {elastic_manager.current_lr:.2e}")
print(f"  New accumulation: {elastic_manager.accumulation_steps}")
print(f"  Global batch: {8 * 2 * elastic_manager.accumulation_steps}")
print()

# Simulate scaling up to 8 GPUs
print("Scaling up: 2 GPUs -> 8 GPUs")
elastic_manager.on_topology_change(8)
print(f"  New LR: {elastic_manager.current_lr:.2e}")
print(f"  New accumulation: {elastic_manager.accumulation_steps}")
print(f"  Global batch: {8 * 8 * elastic_manager.accumulation_steps}")

## Summary

This notebook demonstrated a complete GPT-2 fine-tuning pipeline with `elastic_harness`:

| Feature | Details |
|---------|--------|
| Model | GPT-2 (124M parameters, pretrained) |
| Data | WikiText-2 with GPT-2 tokenization |
| Training | 1000 steps, batch 8 (via accumulation) |
| Learning Rate | 5e-5 with 100-step warmup |
| Checkpointing | Memory + NVMe multi-tier |
| Scaling | SQRT rule for topology changes |
| Output | Coherent, Wikipedia-style text |

**Key differences from the basic demo notebook:**
- Uses **pretrained model** instead of training from scratch
- Produces **coherent text** instead of gibberish
- Lower learning rate (5e-5) appropriate for fine-tuning
- Longer warmup (100 steps) for stable fine-tuning

**elastic_harness components demonstrated:**
- `MemorySnapshotBackend` - Fast in-memory checkpoints
- `NVMeBackend` - Persistent disk checkpoints with cleanup
- `LRScalingManager` - Learning rate warmup and topology scaling
- `ElasticScalingManager` - Unified LR + accumulation management

**For production deployment:**
- Add `S3Backend` for durable cloud checkpoints
- Use `CheckpointManager` for coordinated multi-tier saves
- Deploy etcd for multi-node elastic rendezvous
- Enable chaos testing to validate fault tolerance